# Sesión 1

Contenidos:

- Modelos matemáticos con Julia/JuMP
- Heurísticas constructivas

# Julia/JuMP

La combinación Julia/JuMP permite combinar un lenguaje de programación con un lenguaje de modelización. Esto permite una mejor integración de programa y solver.

Nos limitaremos a usar Julia 0.6 con una versión de JuMP acorde.

Primeramente cargamos JuMP y GLPK que será el solver de programación lineal y programación lineal entera que usaremos en el curso.

In [ ]:
using JuMP
using GLPKMathProgInterface

Un modelo muy simple:

In [ ]:
m = Model(solver = GLPKSolverLP())
@variable(m, 0 <= x <= 10 )
@variable(m, 0 <= y <= 30 )

@objective(m, Max, 5x + 3*y )
@constraint(m, 2x + 3y <= 35.0 )

print(m)

status = solve(m)

println("Status: ",status," Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

Un PL en formato estándar se expresa como:

\begin{align}
\text{minimize} \qquad & c^T x \\
 \text{subject to} \quad \quad & A x = b \\
  & x \geq 0 \\
  & xx \in \mathbb{R}^n
\end{align}


In [ ]:
# DATOS
#-------

c=[-2500; -4000; 0; 0; 0] 
b=[200; 100; 750]
A=[1 0 1 0 0; 0 1 0 1 0; 3 5 0 0 1]
m, n = size(A) # m = filas n = columnas

# DECLARACION DEL MODELO
#------------------------

modelo = Model(solver = GLPKSolverLP())
@variable(modelo, x[1:n] >= 0) # Modela x >=0
for i in 1:m # vamos a construir restricciones una a una
    @constraint(modelo, sum(A[i, j]*x[j] for j in 1:n) == b[i]) # i-ésima restricción 
end 
@objective(modelo,Min,sum(c[i]*x[i] for i in 1:n))
status = solve(modelo) # resolvemos el modelo  
println(getobjectivevalue(modelo))
println(getvalue(x))

# Set Covering Problem

Veamos un ejemplo un poco más complejo en que es importante trabajar sabiendo que la mayoría de la matriz A está formada por ceros.

El modelo es:

\begin{align}
\text{minimize} \qquad & \sum_{j=1}^{n} c_j x_j \\
 \text{subject to} \quad \quad & \sum_{i=1}^{m} a_{ij} x_j \geq 1 & \qquad \forall i\in 1,\cdots,m \\
 \qquad \qquad & x_j \in \{0,1\} & \qquad \forall j\in 1,\cdots,n
\end{align}

Primero vamos a bajar un archivo con datos del problema 

In [ ]:
run(`wget -O scp41.txt http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/scp41.txt`)

In [ ]:
run(`cat scp41.txt`)

La estructura de datos es algo confusa. Primero tenemos una línea con el número de restricciones (200) y en número de variables (1000). Los siguientes 1000 valores corresponden a los costos de las variables en la función objetivo y después hay la descripción de las 200 restricciones con esta forma (copio la primera)

 17 
 91 214 230 289 351 416 488 491 518 567 720 721 
 735 753 768 928 990 
 
 Que significa que hay 17 no ceros y que las variables 91, ..., 990 adoptan valor en la restriccion.
 
 Como la estructura es un tanto oscura, vamos a tener que crear nuestro propio lector de instancias:

In [ ]:
function readFile(filename)
    f=open(filename,"r")
    s=readstring(f)
    close(f)
    s=replace(s,"\n","")
    s=split(s," ")
    nC=parse(Int64,s[2]) #numero de restricciones
    nV=parse(Int64,s[3]) #numero of variables
    numCoefFo=0 #contadores para procesar la información
    numCoef=0
    numRow=0
    c= Float64[] #vector donde guardar los coeficientes de la función objetivo
    #método para almacenar una matriz (o un grafo) sparse
    I=Int64[] #puntero a fila
    J=Int64[] # puntero a columna
    V=Float64[] # puntero a valor
    for i in 4:length(s)
        if s[i]!= ""
            if numCoefFo < nV
                push!(c,parse(Float64,s[i]))
                numCoefFo += 1
            else
                if numCoef == 0
                    numRow += 1
                    numCoef = parse(Int64,s[i])
                else
                    numCoef -= 1
                    push!(I,numRow)
                    push!(J,parse(Int64,s[i]))
                    push!(V,1.0)
                end
            end
        end
    end
    A=sparse(I,J,V) #crea una matriz optimizada para no guardar ceros
    return nC,nV,c,A #,c,A
end

In [ ]:
constraints,variables,c,A = readFile("scp41.txt")

println("\n\n",constraints," ",variables)
println("\n\n",c)
println("\n\n",A)

In [ ]:
scp=Model(solver = GLPKSolverMIP())
@variable(scp, x[1:variables],Bin)

# si se quiere la versión relajada
#scp=Model(solver=GLPKSolverLP())
#@variable(scp, x[1:variables] >= 0)

#función objetivo a minimizar
@objective(scp, Min, sum(c[j]*x[j] for j in 1:variables))

for i in 1:constraints
    @constraint(scp,sum(A[i,j]*x[j] for j in 1:variables) >=1 )
end

#escribimos el modelo
#print(scp)

#resolvemos
status = solve(scp)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(scp))
println("**** x = ", getvalue(x))

In [ ]:
scpAlt=Model(solver = GLPKSolverMIP())
@variable(scpAlt, xAlt[1:variables],Bin)

#función objetivo a minimizar
Expr = AffExpr()
for j in 1:variables
    push!(Expr, c[j], xAlt[j])
end
@objective(scpAlt, Min, Expr)

for i in 1:constraints
    Expr = AffExpr()
    for j in 1:variables
        push!(Expr,A[i,j],xAlt[j])
    end
    @constraint(scpAlt,Expr >= 1)
end

#escribimos el modelo
#print(scpAlt)

#resolvemos
status = solve(scpAlt)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(scpAlt))
for i in 1:variables
    if getvalue(xAlt[i])>0.99
        println(i,"\t",xAlt[i])
    end
end

# Heurísticas constructivas

Recordemos la idea básica de una heurística constructiva: intentar construir la solución paso por paso hasta que tenemos una solución completa.

En este caso necesitamos seleccionar variables hasta cumplir con todas las restricciones. Idealmente queremos "pagar" poco para cumplir con todas las restricciones. 

Empezaremos por definir los datos que vamos a necesitar:

In [ ]:
solucion=zeros(Bool,variables)
cubre=zeros(Int64,variables)
cubierta=zeros(Bool,constraints)
sinCubrir=constraints
costeSolucion=0

In [ ]:
#determinamos el número de restricciones que cubre inicialmente cada variable
#Nota: Hay maneras más eficientes aprovechando "sparse" (https://docs.julialang.org/en/v0.6/manual/arrays/#Sparse-Vectors-and-Matrices-1))

for i in 1:variables
    for j in 1:constraints
        if A[j,i]==1
            cubre[i] += 1
        end
    end
end
#al final de este bucle, cubre[i] indica el número de restricciones que "cubre" la variable i

In [ ]:
while sinCubrir>0
    bestI::Int64=0
    bestCoste=typemax(Float64) #puede igualarse bestCoste a un valor muy alto
    for i in 1:variables
        if solucion[i]==false #si no, no vale la pena y evitamos un problema desde un punto computacional
            costePorRestriccion=c[i]/cubre[i]
            if costePorRestriccion<bestCoste #es el mejor hasta el momento?
                bestCoste=costePorRestriccion
                bestI=i
            end 
        end
    end
    #Aqui tenemos al mejor que es bestI
    @assert bestI!=0 #si pasa esto ha habido un error
    solucion[bestI]=true #actualizamos solución
    costeSolucion+=c[bestI] #actualizamos coste acumulado de la solución en construcción
    for j in 1:constraints #vamos a ver qué restricciones han pasado a estar cubiertas
        if A[j,bestI]==1 && cubierta[j]==false
            cubierta[j]=true #si pasan a estar cubiertas
            sinCubrir -= 1
            for i in 1:variables #vamos a ver qué variables eran las que las cubrian
                if A[j,i]==1
                    cubre[i] -= 1
                end
            end
        end
    end
    @assert cubre[bestI]==0
    println("he escogido ",bestI," quedan por cubrir ",sinCubrir," coste acumulado: ",costeSolucion)
end